# Using a Simple Custom Property Model

This demo will show how a user can utilize a custom property model using the WaterTAP framework. For a guide on how to create this simple property model, see [Creating a Simple Property Model](./creating_a_simple_property_model.ipynb). For documentation on property models that already exist in WaterTAP, see [Property Model Documentation](https://watertap.readthedocs.io/en/latest/technical_reference/property_models/index.html).

## Step 1: Import the necessary functions

In [ ]:
from pyomo.environ import ConcreteModel, assert_optimal_termination
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom
from pyomo.util.check_units import assert_units_consistent
import idaes.core.util.scaling as iscale
from watertap.core.solvers import get_solver

# Imports the property model created in the "Creating a Simple Property Model" Jupyter Notebook
%run creating_a_simple_property_model.ipynb

# To import a custom property model, custom_prop_pack, add it to the following directory and run the line below
# from watertap.property_models.custom_prop_pack import ProcessBlockClassName

## Step 2: Create the ConcreteModel and FlowsheetBlock
Create the flowsheet by touching the properties to build them on the state block and fixing the state variables. Default scaling should also be set for the flow rate to ensure the model is well-scaled.

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Attach property package
m.fs.properties = PropParameterBlock()
# Build a state block, must specify a time which, by convention for steady state models, is just 0
m.fs.stream = m.fs.properties.build_state_block([0])

# Display the state block, it only has the state variables and they are all unfixed
print("\n---First Display---")
m.fs.stream[0].display()

In [ ]:
# Attempt to access properties so that they are built
m.fs.stream[0].mass_frac_phase_comp
# After touching the property, the state block automatically builds it,
# note the mass_frac_phase_comp variable and the constraint to calculate it
print("\n---Second Display---")
m.fs.stream[0].display()

In [ ]:
# Touch another property
m.fs.stream[0].conc_mass_phase_comp
# After touching this property, the state block automatically builds it AND any other properties that are necessary,
# note that now there is the conc_mass_phase_comp and dens_mass_phase variable and associated constraints
print("\n---Third Display---")
m.fs.stream[0].display()

# Touch another property
m.fs.stream[0].flow_vol_phase

# Now that we have a state block, we can fix the state variables and solve for the properties
m.fs.stream[0].temperature.fix(273.15 + 25)
m.fs.stream[0].pressure.fix(101325)
m.fs.stream[0].flow_mass_phase_comp["Liq", "H2O"].fix(1)
m.fs.stream[0].flow_mass_phase_comp["Liq", "NaCl"].fix(0.035)
m.fs.stream[0].flow_mass_phase_comp["Liq", "TSS"].fix(120e-6)

# The user should provide the scale for the flow rate so that our tools can ensure the model is well scaled
# Generally scaling factors should be such that if it is multiplied by the variable it will range between 0.01 and 100
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling(
    "flow_mass_phase_comp", 1e2, index=("Liq", "NaCl")
)
m.fs.properties.set_default_scaling(
    "flow_mass_phase_comp", 1e4, index=("Liq", "TSS")
)
iscale.calculate_scaling_factors(m.fs)  # this utility scales the model

## Step 3: Solve the flowsheet and display results

In [ ]:
# Check that units are consistent
assert_units_consistent(m)

# Check that the degrees of freedom are what we expect
assert (
    degrees_of_freedom(m) == 0
)  

solver = get_solver()
results = solver.solve(m, tee=False)

# Check that the solver finds an optimal solution
assert_optimal_termination(results)

# Display results
print("\n---fourth display---")
m.fs.stream[0].display()
# Note that the properties are solved, and the body of the constraints are small (residual)

## Step 4: Solve the flowsheet and display results under new conditions

In [ ]:
# Equation oriented modeling has several advantages, one of them is that we can unfix variables and fix others
# instead of setting the mass flow rates, we can set the volumetric flow rate and mass fractions
m.fs.stream[0].flow_mass_phase_comp["Liq", "H2O"].unfix()
m.fs.stream[0].flow_mass_phase_comp["Liq", "NaCl"].unfix()
m.fs.stream[0].flow_mass_phase_comp["Liq", "TSS"].unfix()

m.fs.stream[0].flow_vol_phase["Liq"].fix(1.5e-3)
m.fs.stream[0].mass_frac_phase_comp["Liq", "NaCl"].fix(0.05)
m.fs.stream[0].mass_frac_phase_comp["Liq", "TSS"].fix(80e-6)

# Re-solve
results = solver.solve(m, tee=False)
assert_optimal_termination(results)

print("\n---fifth display---")
m.fs.stream[0].display()